In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/SIH2020/GE_Healthcare
!pwd
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/SIH2020/GE_Healthcare
/content/gdrive/My Drive/SIH2020/GE_Healthcare
data				   LSTM.ipynb
Data_Preprocessing_Temporal.ipynb  non_temporal
EDA_Temporal.ipynb		   sepsis.h5
Final_Model			   sepsis_improved.h5
interpolated_BFfill_mean.csv	   sepsis.ipynb
interpolater.csv


In [4]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import os
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
  print('2x')
except Exception:
  pass
import tensorflow as tf
# tf.random.set_seed(1)
import seaborn as sns
from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras import regularizers

2x


In [5]:
model = tf.keras.models.load_model('Final_Model/sepsis_improved.h5')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_4 (LSTM)                (None, 60, 60)            16320     
_________________________________________________________________
lstm_5 (LSTM)                (None, 60, 60)            29040     
_________________________________________________________________
dense_4 (Dense)              (None, 60, 5)             305       
_________________________________________________________________
dense_5 (Dense)              (None, 60, 2)             12        
Total params: 45,677
Trainable params: 45,677
Non-trainable params: 0
_________________________________________________________________


In [64]:
# pre = pd.read_csv('Final_Model/p000053.psv',delimiter='|')
pre = pd.read_csv('Final_Model/p100004.psv',delimiter='|')#no sepesis
length = pre.shape[0]
print(length)

59


In [0]:
pre = pre.interpolate()
pre = pre.fillna(method='bfill')
pre = pre.fillna(method='ffill')
pre = pre.fillna(pre.mean())
# pre.isna().sum()

In [0]:
# X_HR = np.array(pre['HR'])
# X_Temp = np.array(pre['Temp'])
train = pre
X_HR = np.array(pre['HR'])#(no_sampels,60,no_features)
X_Temp = np.array(pre['Temp'])#(no_sampels,60,no_features)
X_Resp = np.array(pre['Resp'])
X_WBC = np.array(pre['WBC'])
X_MAP = np.array(pre['MAP'])
X_Gender = np.array(pre['Gender'])
X_Age = np.array(pre['Age'])
# y = train.groupby('Patient_id')['SepsisLabel_2'].apply(list)
y_original = pre['SepsisLabel']

In [0]:
temp = np.column_stack((X_HR,X_Temp,X_Resp,X_WBC,X_MAP,X_Gender,X_Age))
l = len(temp)

In [0]:
f = ['X_HR','X_Temp','X_Resp','X_WBC','X_MAP','X_Gender','X_Age']

In [0]:
padded_ip = tf.keras.preprocessing.sequence.pad_sequences([temp],maxlen = 60,padding='post',value=-1)
y_pad = tf.keras.preprocessing.sequence.pad_sequences([y_original],maxlen = 60,padding='post',value=-1) 
# print(padded_ip)
ip = padded_ip.reshape(-1, 60, len(f))

In [21]:
ip.shape

(1, 60, 7)

In [63]:
y1 = model.predict(padded_ip)
for i,j in zip(y1[0],y_original):
  print(i,j)

[0.48889637 0.5157368 ] 0
[0.50857747 0.50409615] 0
[0.52032363 0.49686727] 0
[0.52760684 0.49237755] 0
[0.53253794 0.48945016] 0
[0.53617454 0.48743713] 0
[0.53905016 0.4859896 ] 0
[0.5414455 0.4849115] 0
[0.5435161  0.48408583] 0
[0.5453528  0.48343807] 0
[0.54701203 0.48291844] 0
[0.54853034 0.48249155] 0
[0.5499329 0.4821321] 0
[0.55123764 0.48182124] 0
[0.55245763 0.48154515] 0
[0.5536025  0.48129356] 0
[0.5546797 0.481059 ] 0
[0.5556948  0.48083627] 0
[0.556652  0.4806216] 0
[0.5575547  0.48041275] 0
[0.5584054 0.4802083] 0
[0.5592062 0.4800076] 0
[0.5599587  0.47981048] 0
[0.56066453 0.47961712] 0
[0.5613251  0.47942805] 0
[0.5619416  0.47924364] 0
[0.56251556 0.47906476] 0
[0.56304806 0.47889182] 0
[0.56354076 0.4787255 ] 0
[0.56399506 0.47856632] 0
[0.56441253 0.47841468] 0
[0.5647947  0.47827092] 0
[0.5651434 0.4781353] 0
[0.5654604  0.47800794] 0
[0.5657473  0.47788894] 0
[0.56600606 0.47777814] 0
[0.5662384 0.4776756] 0
[0.5664461  0.47758105] 0
[0.5666311  0.47749427] 0
[0

In [47]:
y1 = model.predict(padded_ip)
for i,j in zip(y1[0],y_original):
  print(i,j)

[0.47345033 0.5161647 ] 0
[0.48577178 0.51725054] 0
[0.47250786 0.52453095] 0
[0.48437792 0.5151952 ] 0
[0.4860548 0.5140194] 0
[0.49903822 0.5049168 ] 0
[0.5004536 0.5039245] 0
[0.50220543 0.50269616] 0
[0.51923066 0.490753  ] 0
[0.53850925 0.4772033 ] 0
[0.549908 0.466532] 0
[0.5583249  0.44631678] 0
[0.5676507  0.42654857] 0
[0.5668909  0.41878623] 0
[0.5697723 0.4123046] 0
[0.5790067  0.39875773] 1
[0.6010746  0.37946084] 1
[0.6049754  0.37561822] 1
[0.6094174  0.36977795] 1
[0.62067354 0.35979784] 1
[0.6454863  0.34234157] 1
[0.6492467 0.3384623] 1
[0.6623666 0.3281994] 1
[0.66912514 0.32313043] 1
[0.6743675  0.31865773] 1
